In [1]:
import pandas as pd
import matplotlib.pyplot as plt  # To visualize
import tensorflow as tf
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn import preprocessing, svm


# Read data from csv file 


In [2]:
# Datensatz von https://www.kaggle.com/lehaknarnauli/spotify-datasets?select=tracks.csv
df = pd.read_csv("tracks.csv")

# Preprocessing 

In [3]:
def label_encoding(df):
    encoder = preprocessing.LabelEncoder()
    df['name'] = encoder.fit_transform(df['name'])
    
    return df

df = label_encoding(df)

# Target encoding: encodiere mit durchschnittlicher Popularität des Artists
df['id_artists'] = df['id_artists'].map(df.groupby('id_artists')['popularity'].mean())


df[['year', 'month', 'day']] = df['release_date'].str.split('-', expand=True)
file_name = './preprocessed_tracks.csv'

# drop elemtents that are zero
df.dropna(axis=0, inplace=True)  
df.to_csv(file_name)

# Erklärung für die Features https://rstudio-pubs-static.s3.amazonaws.com/594440_b5a14885d559413ab6e57087eddd68e6.html
# Teilen von df in Daten die Algo sehen darf (X) und das Ergebnis (y)
x_features = ['explicit', 'danceability', 'energy', 'loudness',
              'speechiness', 'acousticness', 'instrumentalness', 'liveness',
              'valence', 'tempo', 'time_signature', 'name', 'id_artists', 'year', 'month', 'day','duration_ms', 'key', 'mode']

#Dimension reduction: 'duration_ms', 'key', 'mode'

X = df[x_features].copy()

y_features = 'popularity'
y = df[y_features].copy()

for col in X.columns:
    X[col] = X[col].astype(float)
    X[col] = (X[col] - X[col].min()) / (X[col].max() - X[col].min())
    
        

# create heatmap

In [ ]:
relevant_features = x_features
relevant_features.append('popularity')

heatmap_df = df[relevant_features]
heatmap_df.sample(frac=1)  # shuffle heatmap_df
heatmap_df = heatmap_df.apply(pd.to_numeric)
for col in heatmap_df.columns:
    heatmap_df[col] = (heatmap_df[col] - heatmap_df[col].min()) / (heatmap_df[col].max() - heatmap_df[col].min())
plt.figure(figsize=(20, 10))
heatmap = sns.heatmap(heatmap_df.corr(), cmap="coolwarm", annot=True)
plt.title("Heatmap: Korrelation von Features", fontsize=20, fontweight="bold")
figure = heatmap.get_figure()
figure.savefig('heatmap.png', dpi=500)

# Splitting Data into train and test set

In [4]:
#  70% into train and 30% into test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, shuffle=True)

# Implementing of metrics and print output

In [5]:
# Define für die Ausgabe von mean_absolute_error auf dem Monitor
def mae(y_pred, y_test, name, name_variable, value_variable):
    mae = mean_absolute_error(y_test, y_pred)
    print(name, name_variable, " =", value_variable, " mean_absolute_error: ", mae)

# Define für die Ausgabe von root_mean_squared_error auf dem Monitor
def rmse(y_pred, y_test, name, name_variable, value_variable):
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    print(name, name_variable, " =", value_variable, " root_mean_squared_error: ", rmse)

# ML-methods

## 1. LinearRegression

In [8]:
clf = LinearRegression()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
mae(y_pred, y_test, "LinearRegression", "", "")
rmse(y_pred, y_test, "LinearRegression", "", "")

#LinearRegression   =   mean_absolute_error:  5.759611991731572
#LinearRegression   =   root_mean_squared_error:  8.437444009357849

LinearRegression   =   mean_absolute_error:  5.824078346487242
LinearRegression   =   root_mean_squared_error:  8.51963224732855


## 2. SupportVectorMachine

In [9]:
clf = svm.SVR()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
mae(y_pred, y_test, "support_vector_machine", "", "")
rmse(y_pred, y_test, "support_vector_machine", "", "")

support_vector_machine   =   mean_absolute_error:  5.705555370089951
support_vector_machine   =   root_mean_squared_error:  8.566518543269583


## 3. RandomForestRegressor

In [10]:
for n_estimators in range(5, 100, 5):
    clf = RandomForestRegressor(n_estimators)
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)
    mae(y_pred, y_test, "random_forest_regressor", "n_estimators", n_estimators)
    rmse(y_pred, y_test, "random_forest_regressor", "n_estimators", n_estimators)

random_forest_regressor n_estimators  = 5  mean_absolute_error:  6.153703981128614
random_forest_regressor n_estimators  = 5  root_mean_squared_error:  8.85809024755132
random_forest_regressor n_estimators  = 10  mean_absolute_error:  5.8343621756121316
random_forest_regressor n_estimators  = 10  root_mean_squared_error:  8.45038834044449
random_forest_regressor n_estimators  = 15  mean_absolute_error:  5.708920268094295
random_forest_regressor n_estimators  = 15  root_mean_squared_error:  8.302250899351247
random_forest_regressor n_estimators  = 20  mean_absolute_error:  5.651329315874791
random_forest_regressor n_estimators  = 20  root_mean_squared_error:  8.234814082129576
random_forest_regressor n_estimators  = 25  mean_absolute_error:  5.597642985837436
random_forest_regressor n_estimators  = 25  root_mean_squared_error:  8.182764564556825
random_forest_regressor n_estimators  = 30  mean_absolute_error:  5.573456840667878
random_forest_regressor n_estimators  = 30  root_mean_squar

## 4. KNeighborsRegressor

In [6]:
for n_neighbors in range(5, 100, 5):
    clf = KNeighborsRegressor(n_neighbors)
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)
    mae(y_pred, y_test, "k_nearest_neighbors", "n_neighbors", n_neighbors)
    rmse(y_pred, y_test, "k_nearest_neighbors", "n_neighbors", n_neighbors)

k_nearest_neighbors n_neighbors  = 5  mean_absolute_error:  7.31867584154733
k_nearest_neighbors n_neighbors  = 5  root_mean_squared_error:  9.877867189806372
k_nearest_neighbors n_neighbors  = 10  mean_absolute_error:  7.091746326948113
k_nearest_neighbors n_neighbors  = 10  root_mean_squared_error:  9.534495670955055
k_nearest_neighbors n_neighbors  = 15  mean_absolute_error:  7.060385097018164
k_nearest_neighbors n_neighbors  = 15  root_mean_squared_error:  9.463540027158908
k_nearest_neighbors n_neighbors  = 20  mean_absolute_error:  7.073355402640877
k_nearest_neighbors n_neighbors  = 20  root_mean_squared_error:  9.457651009556258
k_nearest_neighbors n_neighbors  = 25  mean_absolute_error:  7.096453784638276
k_nearest_neighbors n_neighbors  = 25  root_mean_squared_error:  9.464984230044236
k_nearest_neighbors n_neighbors  = 30  mean_absolute_error:  7.119367925113137
k_nearest_neighbors n_neighbors  = 30  root_mean_squared_error:  9.479039867707527
k_nearest_neighbors n_neighbors

## 5. NeuralNet

In [6]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Input(shape=(19,)))
model.add(tf.keras.layers.Dense(units=64))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dense(units=32))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dense(units=1))
model.add(tf.keras.layers.Activation('linear'))

# Compile the model
model.compile(loss=tf.keras.losses.MeanAbsoluteError(),
              optimizer="adam",
              metrics=['mae', tf.keras.metrics.RootMeanSquaredError()]
              )

history = model.fit(X_train, y_train,
                    batch_size=32,
                    epochs=500,
                    validation_data=(X_test, y_test)
                    )


Epoch 1/500
9802/9802 [==============================] - 12s 1ms/step - loss: 6.2263 - mae: 6.2263 - root_mean_squared_error: 9.4343 - val_loss: 5.7119 - val_mae: 5.7119 - val_root_mean_squared_error: 8.6122
Epoch 2/500
9802/9802 [==============================] - 11s 1ms/step - loss: 5.7126 - mae: 5.7126 - root_mean_squared_error: 8.5799 - val_loss: 5.6819 - val_mae: 5.6819 - val_root_mean_squared_error: 8.5616
Epoch 3/500
9802/9802 [==============================] - 11s 1ms/step - loss: 5.6943 - mae: 5.6943 - root_mean_squared_error: 8.5713 - val_loss: 5.6633 - val_mae: 5.6633 - val_root_mean_squared_error: 8.5681
Epoch 4/500
9802/9802 [==============================] - 11s 1ms/step - loss: 5.6843 - mae: 5.6843 - root_mean_squared_error: 8.5696 - val_loss: 5.6563 - val_mae: 5.6563 - val_root_mean_squared_error: 8.5436
Epoch 5/500
9802/9802 [==============================] - 11s 1ms/step - loss: 5.6748 - mae: 5.6748 - root_mean_squared_error: 8.5649 - val_loss: 5.6553 - val_mae: 5.655

9802/9802 [==============================] - 12s 1ms/step - loss: 5.5944 - mae: 5.5944 - root_mean_squared_error: 8.5322 - val_loss: 5.6034 - val_mae: 5.6034 - val_root_mean_squared_error: 8.5086
Epoch 41/500
9802/9802 [==============================] - 12s 1ms/step - loss: 5.5921 - mae: 5.5921 - root_mean_squared_error: 8.5316 - val_loss: 5.6055 - val_mae: 5.6055 - val_root_mean_squared_error: 8.5284
Epoch 42/500
9802/9802 [==============================] - 12s 1ms/step - loss: 5.5916 - mae: 5.5916 - root_mean_squared_error: 8.5320 - val_loss: 5.6036 - val_mae: 5.6036 - val_root_mean_squared_error: 8.5464
Epoch 43/500
9802/9802 [==============================] - 11s 1ms/step - loss: 5.5903 - mae: 5.5903 - root_mean_squared_error: 8.5306 - val_loss: 5.6240 - val_mae: 5.6240 - val_root_mean_squared_error: 8.5134
Epoch 44/500
9802/9802 [==============================] - 12s 1ms/step - loss: 5.5905 - mae: 5.5905 - root_mean_squared_error: 8.5303 - val_loss: 5.5983 - val_mae: 5.5983 - val_

9802/9802 [==============================] - 11s 1ms/step - loss: 5.5709 - mae: 5.5709 - root_mean_squared_error: 8.5205 - val_loss: 5.6089 - val_mae: 5.6089 - val_root_mean_squared_error: 8.5081
Epoch 80/500
9802/9802 [==============================] - 11s 1ms/step - loss: 5.5713 - mae: 5.5713 - root_mean_squared_error: 8.5202 - val_loss: 5.5869 - val_mae: 5.5869 - val_root_mean_squared_error: 8.5239
Epoch 81/500
9802/9802 [==============================] - 11s 1ms/step - loss: 5.5692 - mae: 5.5692 - root_mean_squared_error: 8.5182 - val_loss: 5.5985 - val_mae: 5.5985 - val_root_mean_squared_error: 8.5026
Epoch 82/500
9802/9802 [==============================] - 11s 1ms/step - loss: 5.5692 - mae: 5.5692 - root_mean_squared_error: 8.5194 - val_loss: 5.5940 - val_mae: 5.5940 - val_root_mean_squared_error: 8.5498
Epoch 83/500
9802/9802 [==============================] - 11s 1ms/step - loss: 5.5690 - mae: 5.5690 - root_mean_squared_error: 8.5199 - val_loss: 5.5926 - val_mae: 5.5926 - val_

9802/9802 [==============================] - 11s 1ms/step - loss: 5.5573 - mae: 5.5573 - root_mean_squared_error: 8.5090 - val_loss: 5.6029 - val_mae: 5.6029 - val_root_mean_squared_error: 8.5500
Epoch 119/500
9802/9802 [==============================] - 11s 1ms/step - loss: 5.5575 - mae: 5.5575 - root_mean_squared_error: 8.5078 - val_loss: 5.5779 - val_mae: 5.5779 - val_root_mean_squared_error: 8.5235
Epoch 120/500
9802/9802 [==============================] - 11s 1ms/step - loss: 5.5563 - mae: 5.5563 - root_mean_squared_error: 8.5067 - val_loss: 5.6106 - val_mae: 5.6106 - val_root_mean_squared_error: 8.4882
Epoch 121/500
9802/9802 [==============================] - 11s 1ms/step - loss: 5.5559 - mae: 5.5559 - root_mean_squared_error: 8.5050 - val_loss: 5.5798 - val_mae: 5.5798 - val_root_mean_squared_error: 8.5163
Epoch 122/500
9802/9802 [==============================] - 11s 1ms/step - loss: 5.5563 - mae: 5.5563 - root_mean_squared_error: 8.5048 - val_loss: 5.6165 - val_mae: 5.6165 - 

9802/9802 [==============================] - 12s 1ms/step - loss: 5.5469 - mae: 5.5469 - root_mean_squared_error: 8.4961 - val_loss: 5.5724 - val_mae: 5.5724 - val_root_mean_squared_error: 8.5029
Epoch 158/500
9802/9802 [==============================] - 11s 1ms/step - loss: 5.5460 - mae: 5.5460 - root_mean_squared_error: 8.4963 - val_loss: 5.6093 - val_mae: 5.6093 - val_root_mean_squared_error: 8.5521
Epoch 159/500
9802/9802 [==============================] - 11s 1ms/step - loss: 5.5466 - mae: 5.5466 - root_mean_squared_error: 8.4932 - val_loss: 5.5746 - val_mae: 5.5746 - val_root_mean_squared_error: 8.5270
Epoch 160/500
9802/9802 [==============================] - 11s 1ms/step - loss: 5.5466 - mae: 5.5466 - root_mean_squared_error: 8.4956 - val_loss: 5.5963 - val_mae: 5.5963 - val_root_mean_squared_error: 8.5047
Epoch 161/500
9802/9802 [==============================] - 11s 1ms/step - loss: 5.5459 - mae: 5.5459 - root_mean_squared_error: 8.4940 - val_loss: 5.5836 - val_mae: 5.5836 - 

...

9802/9802 [==============================] - 11s 1ms/step - loss: 5.5218 - mae: 5.5218 - root_mean_squared_error: 8.4738 - val_loss: 5.5858 - val_mae: 5.5858 - val_root_mean_squared_error: 8.4854
Epoch 314/500
9802/9802 [==============================] - 11s 1ms/step - loss: 5.5224 - mae: 5.5224 - root_mean_squared_error: 8.4750 - val_loss: 5.5703 - val_mae: 5.5703 - val_root_mean_squared_error: 8.5056
Epoch 315/500
9802/9802 [==============================] - 11s 1ms/step - loss: 5.5210 - mae: 5.5210 - root_mean_squared_error: 8.4732 - val_loss: 5.5733 - val_mae: 5.5733 - val_root_mean_squared_error: 8.4993
Epoch 316/500
9802/9802 [==============================] - 11s 1ms/step - loss: 5.5214 - mae: 5.5214 - root_mean_squared_error: 8.4722 - val_loss: 5.5681 - val_mae: 5.5681 - val_root_mean_squared_error: 8.4992
Epoch 317/500
9802/9802 [==============================] - 11s 1ms/step - loss: 5.5212 - mae: 5.5212 - root_mean_squared_error: 8.4742 - val_loss: 5.5680 - val_mae: 5.5680 - 

9802/9802 [==============================] - 11s 1ms/step - loss: 5.5186 - mae: 5.5186 - root_mean_squared_error: 8.4699 - val_loss: 5.5658 - val_mae: 5.5658 - val_root_mean_squared_error: 8.5019
Epoch 353/500
9802/9802 [==============================] - 11s 1ms/step - loss: 5.5180 - mae: 5.5180 - root_mean_squared_error: 8.4707 - val_loss: 5.5653 - val_mae: 5.5653 - val_root_mean_squared_error: 8.5196
Epoch 354/500
9802/9802 [==============================] - 11s 1ms/step - loss: 5.5178 - mae: 5.5178 - root_mean_squared_error: 8.4693 - val_loss: 5.5633 - val_mae: 5.5633 - val_root_mean_squared_error: 8.5003
Epoch 355/500
9802/9802 [==============================] - 11s 1ms/step - loss: 5.5193 - mae: 5.5193 - root_mean_squared_error: 8.4703 - val_loss: 5.5865 - val_mae: 5.5865 - val_root_mean_squared_error: 8.5033
Epoch 356/500
9802/9802 [==============================] - 11s 1ms/step - loss: 5.5178 - mae: 5.5178 - root_mean_squared_error: 8.4698 - val_loss: 5.5819 - val_mae: 5.5819 - 

9802/9802 [==============================] - 11s 1ms/step - loss: 5.5143 - mae: 5.5143 - root_mean_squared_error: 8.4661 - val_loss: 5.5593 - val_mae: 5.5593 - val_root_mean_squared_error: 8.5120
Epoch 392/500
9802/9802 [==============================] - 11s 1ms/step - loss: 5.5153 - mae: 5.5153 - root_mean_squared_error: 8.4671 - val_loss: 5.5696 - val_mae: 5.5696 - val_root_mean_squared_error: 8.4713
Epoch 393/500
9802/9802 [==============================] - 11s 1ms/step - loss: 5.5136 - mae: 5.5136 - root_mean_squared_error: 8.4655 - val_loss: 5.5665 - val_mae: 5.5665 - val_root_mean_squared_error: 8.4847
Epoch 394/500
9802/9802 [==============================] - 11s 1ms/step - loss: 5.5150 - mae: 5.5150 - root_mean_squared_error: 8.4683 - val_loss: 5.5663 - val_mae: 5.5663 - val_root_mean_squared_error: 8.5227
Epoch 395/500
9802/9802 [==============================] - 11s 1ms/step - loss: 5.5141 - mae: 5.5141 - root_mean_squared_error: 8.4655 - val_loss: 5.5610 - val_mae: 5.5610 - 

9802/9802 [==============================] - 11s 1ms/step - loss: 5.5100 - mae: 5.5100 - root_mean_squared_error: 8.4632 - val_loss: 5.5591 - val_mae: 5.5591 - val_root_mean_squared_error: 8.4945
Epoch 431/500
9802/9802 [==============================] - 11s 1ms/step - loss: 5.5109 - mae: 5.5109 - root_mean_squared_error: 8.4628 - val_loss: 5.5631 - val_mae: 5.5631 - val_root_mean_squared_error: 8.4976
Epoch 432/500
9802/9802 [==============================] - 11s 1ms/step - loss: 5.5101 - mae: 5.5101 - root_mean_squared_error: 8.4629 - val_loss: 5.5612 - val_mae: 5.5612 - val_root_mean_squared_error: 8.5337
Epoch 433/500
9802/9802 [==============================] - 11s 1ms/step - loss: 5.5100 - mae: 5.5100 - root_mean_squared_error: 8.4636 - val_loss: 5.5699 - val_mae: 5.5699 - val_root_mean_squared_error: 8.4859
Epoch 434/500
9802/9802 [==============================] - 11s 1ms/step - loss: 5.5109 - mae: 5.5109 - root_mean_squared_error: 8.4630 - val_loss: 5.5751 - val_mae: 5.5751 - 

9802/9802 [==============================] - 11s 1ms/step - loss: 5.5080 - mae: 5.5080 - root_mean_squared_error: 8.4600 - val_loss: 5.5661 - val_mae: 5.5661 - val_root_mean_squared_error: 8.5159
Epoch 470/500
9802/9802 [==============================] - 11s 1ms/step - loss: 5.5073 - mae: 5.5073 - root_mean_squared_error: 8.4591 - val_loss: 5.5661 - val_mae: 5.5661 - val_root_mean_squared_error: 8.4997
Epoch 471/500
9802/9802 [==============================] - 11s 1ms/step - loss: 5.5066 - mae: 5.5066 - root_mean_squared_error: 8.4593 - val_loss: 5.5607 - val_mae: 5.5607 - val_root_mean_squared_error: 8.4814
Epoch 472/500
9802/9802 [==============================] - 11s 1ms/step - loss: 5.5087 - mae: 5.5087 - root_mean_squared_error: 8.4627 - val_loss: 5.5632 - val_mae: 5.5632 - val_root_mean_squared_error: 8.4858
Epoch 473/500
9802/9802 [==============================] - 11s 1ms/step - loss: 5.5065 - mae: 5.5065 - root_mean_squared_error: 8.4590 - val_loss: 5.5769 - val_mae: 5.5769 - 